### 导入相关程序库

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### 获取KPI名称和KPI ID对应关系

In [2]:
kpi_id_path = r'D:\userdata\anliu\Desktop'
kpi_id_name = r'kpi_id_17a.csv'
kpi_id_file = os.path.join(kpi_id_path, kpi_id_name)

try:
    os.chdir(kpi_id_path)
except:
    print("The folder does not existed!")
    
kpi_def = pd.read_csv(kpi_id_file)
kpi_def.dropna(axis=1, how='all', inplace=True)

def query_kpi_id(kpi_id):   
    mask = kpi_def.loc[kpi_def.loc[:, 'KPI ID'] == kpi_id]
    
    kpi_name = mask['Name'].values[0]
    kpi_formula = mask['Formula'].values[0]
    kpi_level = mask['Level'].values[0]
    
    return kpi_id, kpi_name, kpi_formula, kpi_level

### 分析KPI数据

In [3]:
# kpi log所在文件路径
kpi_data_path = r'D:\userdata\anliu\Desktop'
kpi_data_name = r'test.csv'
kpi_data_file = os.path.join(kpi_data_path, kpi_data_name)

# 切换到kpi log所在路径
try:
    os.chdir(kpi_data_path)
except:
    print("The folder does not existed!")

#### 读取eNB软件版本

In [4]:
# 读取csv文件，返回有效数据起始行数
def fetch_sw_version(file):
    nrow = 0
    with open(file,"r") as f:
        sw_version = f.readline()
        sw_version = sw_version.strip()
        sw_version = sw_version.split(',')[0]
    
    return sw_version

In [5]:
print(fetch_sw_version(kpi_data_file))

TL17A_ENB_0000_040029_000000


#### 读取KPI数据

In [6]:
# 从CSV文件中读取数据并存储为dataframe
def read_kpi_data(file):
    kpi_data = pd.read_csv(file, low_memory=True, skiprows=3, error_bad_lines=False, warn_bad_lines=False)
    return kpi_data

# 整理数据
def clean_up_data(data):  
    # 替换第一列名称为Time
    names = kpi_data.columns.tolist()
    names[names.index('Period start time')] = 'Time'
    kpi_data.columns = names
    
    # 删除值全部为NA的列
    kpi_data.dropna(axis=1, how='all', inplace=True)
    
    kpi_data['Time'] = pd.to_datetime(kpi_data['Time'])
    
    kpi_data.set_index('Time', inplace=True)

    return kpi_data

#### 整理数据，获取数据基本信息

In [7]:
# 读取整理数据
kpi_data = read_kpi_data(kpi_data_file)
kpi_data = clean_up_data(kpi_data)
print(kpi_data.info())
print(kpi_data.describe())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336 entries, 2018-06-15 20:00:00 to 2018-06-19 07:45:00
Data columns (total 19 columns):
LTE_5017a    336 non-null float64
LTE_5025h    336 non-null float64
LTE_5137a    336 non-null int64
LTE_5134a    336 non-null float64
LTE_5216a    336 non-null float64
LTE_5441b    336 non-null float64
LTE_5444b    336 non-null float64
LTE_5541b    336 non-null float64
LTE_5544b    336 non-null float64
LTE_5427c    336 non-null float64
LTE_5432b    336 non-null float64
LTE_5292d    336 non-null float64
LTE_5289d    336 non-null float64
LTE_5276b    336 non-null float64
LTE_5273b    336 non-null float64
LTE_6265a    336 non-null int64
LTE_5646a    336 non-null float64
LTE_5471a    336 non-null int64
LTE_5218f    336 non-null int64
dtypes: float64(15), int64(4)
memory usage: 52.5 KB
None
        LTE_5017a   LTE_5025h  LTE_5137a   LTE_5134a   LTE_5216a   LTE_5441b  \
count  336.000000  336.000000      336.0  336.000000  336.000000  336.000000   
mea

#### 打印出KPI ID对应的KPI名字

In [8]:
def read_kpi_name(columns):
    for kpi_id in columns:
        try:
            kpi = query_kpi_id(kpi_id)
            print(kpi_id + ": " + kpi[1])
        except:
            print(kpi_id + ': does not existed!')

read_kpi_name(kpi_data.columns)

LTE_5017a: E-UTRAN E-RAB Setup Success Ratio
LTE_5025h: E-UTRAN E-RAB Drop Ratio, RAN View
LTE_5137a: E-UTRAN Average Latency Uplink
LTE_5134a: E-UTRAN Average Latency Downlink
LTE_5216a: E-UTRAN MAC PDU Re-transmission Ratio Downlink
LTE_5441b: E-UTRAN Average RSSI for PUCCH
LTE_5444b: Average RSSI for PUSCH
LTE_5541b: E-UTRAN Average SINR for PUCCH
LTE_5544b: E-UTRAN Average SINR for PUSCH
LTE_5427c: E-UTRAN Average CQI
LTE_5432b: E-UTRAN Average CQI Offset
LTE_5292d: E-UTRAN average PDCP Layer Active Cell Throughput DL
LTE_5289d: E-UTRAN average PDCP Layer Active Cell Throughput UL
LTE_5276b: E-UTRAN average PRB usage per TTI DL
LTE_5273b: E-UTRAN Average PRB usage per TTI UL
LTE_6265a: E-UTRAN Maximum Number of RRC Connected UEs per cell
LTE_5646a: E-UTRAN E-RAB Setup Success Ratio, QCI5
LTE_5471a: E-UTRAN Average PDCP SDU Delay in DL, QCI1
LTE_5218f: does not existed!


#### 计算相关系数矩阵

In [9]:
def calc_corr(dataframe):
    corr = dataframe.corr()
    corr.dropna(axis=(0,1), how='all', inplace=True)
    sns.heatmap(corr, linewidths = 0.05);
    
    return corr
# np.fill_diagonal(corr.values, np.nan)
# mask = corr>0.6
# corr = corr[mask]
# fig = plt.figure(figsize=(12,8))
# sns.heatmap(corr, linewidths = 0.05);
# corr.dropna(axis=(0,1), how='all', inplace=True)
# print(corr)

#### 分析数据

In [10]:
# 定义KPI组合
cell_load = ['LTE_6245a', 'LTE_6265a', 'LTE_5214b', 'LTE_5804c', 'LTE_5242b', 'LTE_5394a', 'LTE_6001a', 'LTE_6002a',
             'LTE_5319c', 'LTE_5320b', 'LTE_5672b', 'LTE_5674b', 'LTE_5837b', 'LTE_5838b', 'LTE_5999a', 'LTE_753c', 
             'LTE_5320b', 'LTE_5656b', ]
cell_meas = ['LTE_5368a', 'LTE_5369a', 'LTE_5370a', 'LTE_5371a', 'LTE_5427c', 'LTE_5432b', 'LTE_1552b', 'LTE_1553a',
             'LTE_5273b', 'LTE_5276b', 'LTE_5657a']
cplane = ['LTE_5017a', 'LTE_5218g', 'LTE_5204c', 'LTE_5693a', 'LTE_5025h', 'LTE_5572f',
          'LTE_5035a', 'LTE_5568a', 'LTE_5114a', 'LTE_5048b', 'LTE_5873a', 'LTE_5886a', 'LTE_5082a', 'LTE_5884a', 
          'LTE_5880a', 'LTE_5882a', 'LTE_5567a']
uplane = ['LTE_5292d', 'LTE_5289d', 'LTE_5276b', 'LTE_5273b', 'LTE_1074a', 'LTE_1075a']

# 根据counter提取数据
def kpi_filter_counter(kpi_data, comb_list):
    read_kpi_name(comb_list)
    kpi_data = kpi_data.loc[:, comb_list]
    kpi_data.dropna(axis=1, how='any', inplace=True)
    
    return kpi_data

# 根据时间提取数据
def kpi_filter_time(kpi_data, start, end):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    kpi_data = kpi_data.loc[start:end, :]
    
    return kpi_data

In [11]:
# 均值方差检测
def outlier_detect_std(sample, data):
    upper = np.mean(sample) + 3*np.std(sample)
    lower = np.mean(sample) - 3*np.std(sample)
    
    if (data>upper) or (data<lower):
        return False
    else:
        return True

def kpi_outlier_std(kpi_data):
    print('=' * 60)
    
    n = 0
    outlier = {}
    for i in kpi_data.columns:
        f = True
        for j, elem in enumerate(kpi_data[i]):
            flag = outlier_detect_std(kpi_data[i], elem)       
            if flag == False:
                if f:
                    f = False
#                     print(i + ':' + '-'*40)
                outlier[j] = outlier.get(j, 0) + 1
#                 print("{0}th: {1} out of range!".format(str(j), str(elem)))

    outlier_list = sorted(outlier.items(), key=lambda e:e[1], reverse=True)             
    print([x[0] for x in outlier_list])
                
# LOF异常值检测
from sklearn.neighbors import LocalOutlierFactor
def kpi_outlier_lof(kpi_data):
    print('LOF prediction:')
    clf = LocalOutlierFactor(n_neighbors=50, contamination=0.1)
    y_pred = clf.fit_predict(kpi_data)    
    
    print([i for i, e in enumerate(y_pred) if e==-1])
    

# Isolation forest异常值检测
from sklearn.ensemble import IsolationForest
def kpi_outlier_isoforest(kpi_data):
    print('Iosforest prediction:')
    clf = IsolationForest(n_estimators=100, n_jobs=-1, contamination=0.1)
    clf.fit(kpi_data)

    shape = kpi_data.shape[0]
    batch = 10**4

    y_pred = []
    for i in range(int(shape/batch)+1):
        start = i * batch
        end = (i+1) * batch
        test = kpi_data[:][start:end]
    
        pred = clf.predict(test)
        y_pred.extend(pred)

        print([i for i, e in enumerate(y_pred) if e==-1])

D:\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [12]:
start = '2018/6/15  21:00'
end = '2018-06-16 10:0:0'
kpi_data_time = kpi_filter_time(kpi_data, start, end)

# print(kpi_data.head())
kpi_outlier_std(kpi_data_time)

kpi_outlier_lof(kpi_data_time)

kpi_outlier_isoforest(kpi_data_time)

[17, 2, 16, 28, 49, 44]
LOF prediction:
[4, 16, 33, 34, 41, 44]
Iosforest prediction:
[0, 2, 16, 17, 24, 29]
